In [24]:
# start docker deamon in terminal
# !sudo docker login
# OR - becuase the jupyther with allow root
# !docker login

In [25]:
!pwd

/home/eila/git_igvf/atomic-workflows/nextflow/nf_workflow/workflow_fg_single_cell_atac


TODO:

when running with cloud. 
if you dont need to keep the execution logs, change the cloud settings to keep it for a short while
Check that the cache if you need to set the cache to false (especially in plot process)

In [26]:
import os
json_path = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())),"hai-gcp-genomic-beb9207a40b8.json")
!ls $json_path

/home/eila/git_igvf/atomic-workflows/nextflow/hai-gcp-genomic-beb9207a40b8.json


In [27]:
# # in the command line. same env
import os
json_path = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())),"hai-gcp-genomic-beb9207a40b8.json")
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = json_path
!gcloud auth activate-service-account eila-batch@hai-gcp-genomic.iam.gserviceaccount.com --key-file=$GOOGLE_APPLICATION_CREDENTIALS

Activated service account credentials for: [eila-batch@hai-gcp-genomic.iam.gserviceaccount.com]


In [28]:
!gcloud auth list
project_name = !gcloud config get-value project
project_name = project_name[0]
project_name
# gcloud config set account


                  Credentialed Accounts
ACTIVE  ACCOUNT
        886987709752-compute@developer.gserviceaccount.com
*       eila-batch@hai-gcp-genomic.iam.gserviceaccount.com

To set the active account, run:
    $ gcloud config set account `ACCOUNT`



'hai-gcp-genomic'

In [29]:
# !gsutil cp gs://eila-nextflow-bucket/work/ATAC/b3/94bc73c6d43de41d17d75cac4fbda1/no-singleton.bed.gz.tbi /home/eila/git_igvf/downloads

In [30]:
import os
os.environ["TOWER_ACCESS_TOKEN"] = 'eyJ0aWQiOiA4Mjk3fS5jNDNmYzA4NTg2MjAxNDQzOTAwYmUzNWEzZTY0MGMyZTc0OWE3M2Mz'
os.environ["TOWER_WORKSPACE_ID"]='247507587970818'


In [31]:
!echo $TOWER_ACCESS_TOKEN
!echo $TOWER_WORKSPACE_ID

eyJ0aWQiOiA4Mjk3fS5jNDNmYzA4NTg2MjAxNDQzOTAwYmUzNWEzZTY0MGMyZTc0OWE3M2Mz
247507587970818


In [32]:
!pwd

/home/eila/git_igvf/atomic-workflows/nextflow/nf_workflow/workflow_fg_single_cell_atac


In [33]:
# !chmod +x ./bin/scrna_qc_metrics.py
import os
bin_fld = os.path.join(os.getcwd(),"bin")
bin_fld

'/home/eila/git_igvf/atomic-workflows/nextflow/nf_workflow/workflow_fg_single_cell_atac/bin'

In [34]:
!chmod +x {os.path.join(bin_fld, "atac_generate_metadata_barcode_rank_plot.R")}
!chmod +x {os.path.join(bin_fld, "run_tabix.sh")}
!chmod +x {os.path.join(bin_fld, "atac_generate_merged_barcode_metadata.sh")}
!chmod +x {os.path.join(bin_fld, "create_sample_filtered_fragment_file.sh")}
!chmod +x {os.path.join(bin_fld, "compute_tss_enrichment_bulk.py")}
!chmod +x {os.path.join(bin_fld, "barcode_rank_functions.R")}
!chmod +x {os.path.join(bin_fld, "plot_insert_size_hist.py")}
!chmod +x {os.path.join(bin_fld, "compute_tss_enrichment_snapatac2.py")}
!chmod +x {os.path.join(bin_fld, "add_subpool_to_chromap_output.sh")}
!chmod +x {os.path.join(bin_fld, "add_subpool_to_summary_file.sh")}
!chmod +x {os.path.join(bin_fld, "filter_fragments_gz_sort_bgzip_script.sh")}
!chmod +x {os.path.join(bin_fld, "merge_chromap_logs.sh")}
!chmod +x {os.path.join(bin_fld, "generate_insert_size_histogram_data.sh")}
!chmod +x {os.path.join(bin_fld, "calculate_barcode_metadata_stats.sh")}

In [35]:
# # TODO: For a new user: run these commands with your service account for the execution. I am using eila-batch@...

# !gcloud projects add-iam-policy-binding hai-gcp-genomic \
# --member="serviceAccount:eila-batch@hai-gcp-genomic.iam.gserviceaccount.com" \
# --role="roles/iam.serviceAccountUser"

# !gcloud projects add-iam-policy-binding hai-gcp-genomic \
# --member="serviceAccount:eila-batch@hai-gcp-genomic.iam.gserviceaccount.com" \
# --role="roles/batch.jobsEditor"

# !gcloud projects add-iam-policy-binding hai-gcp-genomic \
# --member="serviceAccount:eila-batch@hai-gcp-genomic.iam.gserviceaccount.com" \
# --role="roles/logging.viewer"

# !gcloud projects add-iam-policy-binding hai-gcp-genomic \
# --member="serviceAccount:eila-batch@hai-gcp-genomic.iam.gserviceaccount.com" \

In [36]:
# !nextflow -self-update

In [37]:
!nextflow info

  Version: 23.10.1 build 5891
  Created: 12-01-2024 22:01 UTC 
  System: Linux 5.10.0-27-cloud-amd64
  Runtime: Groovy 3.0.19 on OpenJDK 64-Bit Server VM 11.0.1-internal+0-adhoc..src
  Encoding: UTF-8 (UTF-8)



In [38]:
import os

# Get the current working directory
current_directory = os.getcwd()

# Navigate two folders above
two_folders_above = os.path.abspath(os.path.join(current_directory, '..', '..'))

nf_data_base_path = os.path.join(two_folders_above,'nf_data')
# Print the result
print(nf_data_base_path)


/home/eila/git_igvf/atomic-workflows/nextflow/nf_data


In [39]:
nf_data_base_path


'/home/eila/git_igvf/atomic-workflows/nextflow/nf_data'

In [40]:
# nf_data_project="test_atac_pipeline"
nf_data_project="wdl_atac_pipeline"

In [41]:
# LOCAL
# os.path.exists(os.path.join(nf_data_base_path,nf_data_project,"local_atac_fastqs_files.tsv"))
# os.path.join(nf_data_base_path,nf_data_project,"local_atac_fastqs_files.tsv")

# VM
os.path.exists(os.path.join(nf_data_base_path,nf_data_project,"vm_atac_fastqs_files.tsv"))
os.path.join(nf_data_base_path,nf_data_project,"vm_atac_fastqs_files.tsv")

'/home/eila/git_igvf/atomic-workflows/nextflow/nf_data/wdl_atac_pipeline/vm_atac_fastqs_files.tsv'

In [42]:
# !cp ../../config_files/docker-google-batch.config ../workflow_fg_single_cell_rna/conf/

In [43]:

# Write the configuration file
with open("nextflow_atac.config", "w") as config_file:
    config_file.write(f"""
manifest {{
  description = 'A nexflow pipeline for fg single cell atac' 
  mainScript = 'nf_wf_fg_singlecell_atac.nf'
}}

profiles {{
  google_batch_profile {{
    workDir = 'gs://eila-nextflow-bucket/work/ATAC'
    includeConfig 'conf/docker-google-batch.config'
    docker.enabled = true
    wave.enabled = true
    fusion.enabled = true
    process.scratch = false
    executor = 'google-batch'
    docker.envWhitelist=['ENV_SYNAPSE_TOKEN','TOWER_ACCESS_TOKEN']
    google.project = '{project_name}'
    google.location = 'us-central1'
    google.batch.spot = true
    google.batch.serviceAccountEmail='eila-batch@hai-gcp-genomic.iam.gserviceaccount.com'
  }}
}}

// Define ENV_SYNAPSE_TOKEN as a parameter
params.FASTQS_SPEC_CH = '{nf_data_base_path}/{nf_data_project}/vm_atac_fastqs_files.tsv'
// CHROMAP
// 6a
params.CHROMAP_IDX='gs://eila-nextflow-bucket/test_atac_pipeline/ref.index'
// 6b:
// FASTA file
params.CHROMAP_READ_LENGTH = 2000
params.CHROMAP_BC_ERROR_THRESHOLD = 2
params.CHROMAP_BC_PROBABILITY_THRESHOLD = 0.9
params.CHROMAP_READ_FORMAT = 'bc:0:-1,r1:0:-1,r2:0:-1'
params.CHROMAP_DROP_REPETITIVE_READS = 4
params.CHROMAP_QUALITY_THRESHOLD = 0
params.CHROMAP_MERGE_LOGS_SCRIPT = "merge_chromap_logs.sh"
params.CHROMAP_FRAGMENTS_HISTOGRAM_DATA_SCRIPT='generate_insert_size_histogram_data.sh'
params.CHROMAP_GENOME_REFERENCE_FASTA='{nf_data_base_path}/{nf_data_project}/GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.gz'

// CHROMAP DEBUG: load output files 
// params.chromap_filter_fragments_tsv ='{nf_data_base_path}/{nf_data_project}/chromap_out_debug/ATAC_BMMC-single-donor.atac.filter.fragments.tsv'
// params.barcode_summary_csv ='{nf_data_base_path}/{nf_data_project}/chromap_out_debug/BMMC-single-donor.atac.align.barcode.summary.csv'
// params.chromap_alignment_log_out = '{nf_data_base_path}/{nf_data_project}/chromap_out_debug/BMMC-single-donor.atac.align.k4.hg38.log.txt'

// CHROMAP WDL DEBUG: load output files 
params.chromap_fragments_tsv ='{nf_data_base_path}/{nf_data_project}/ATAC_BMMC-single-donor.atac.filter.fragments.tsv'
params.barcode_summary_csv ='{nf_data_base_path}/{nf_data_project}/BMMC-single-donor.atac.align.barcode.summary.csv'
params.chromap_alignment_log_out = '{nf_data_base_path}/{nf_data_project}/BMMC-single-donor.atac.align.k4.hg38.log.txt'
params.bgzip_subpool_chromap_fragments_tsv_out='{nf_data_base_path}/{nf_data_project}/BMMC-single-donor.atac.filter.fragments.hg38.tsv.gz'
params.no_singleton_gz='{nf_data_base_path}/{nf_data_project}/no-singleton.bed.gz'
// params.tbi_no_singleton_bed_gz_out='{nf_data_base_path}/{nf_data_project}/.tbi'

params.TSV_ADD_SUBPOOL_SCRIPT = 'add_subpool_to_chromap_output.sh'
params.CSV_ADD_SUBPOOL_SCRIPT = 'add_subpool_to_summary_file.sh'
params.FILTER_FRAGMENTS_SCRIPT = 'filter_fragments_gz_sort_bgzip_script.sh'
params.ATAC_FRAGMENTS_MIN_FRAG_CUTOFF=10

// Other steps

params.GENOME_GZ_GTF='http://ftp.ensembl.org/pub/release-109/gtf/homo_sapiens/Homo_sapiens.GRCh38.109.gtf.gz'
params.ENV_SYNAPSE_TOKEN = "nextflow secrets get SYNAPSE".execute().text.trim()


params.MAPQ_THRESHOLD = false
params.BARCODE_TRANSLATE = 'path'
params.ATAC_FRAGMENTS_CUTOFF = 4
params.RUN_TABIX_SCRIPT='run_tabix.sh'
params.ATAC_TSS_BASES_FLANK = 2000
params.ATAC_TSS_COL_WITH_STRANDS_INFO = 4
params.ATAC_TSS_SMOOTHING_WINDOW_SIZE = 20
params.ATAC_TSS_REGION_BED_FILE='{nf_data_base_path}/{nf_data_project}/hg38.refGene.TSS.bed'
params.ATAC_TSS_BULK_CALCULATION_SCRIPT='compute_tss_enrichment_bulk.py'
params.ATAC_TSS_SNAPATAC2_CALCULATION_SCRIPT='compute_tss_enrichment_snapatac2.py'
params.ATAC_TSS_SNAPATAC2_GENES_GTF='{nf_data_base_path}/{nf_data_project}/genes.gtf'
params.ATAC_TSS_SNAPATAC2_MIN_FRAG_CUTOFF = 10

params.ATAC_INSERT_SIZE_PLOT_SCRIPT='plot_insert_size_hist.py'
params.pkr_id="BMM_PKR_ID"

// atac calculate_barcode_metadata_stats
params.ATAC_SNAP_METADATA_STATS_SCRIPT='calculate_barcode_metadata_stats.sh'
// atac plot barcode metadata
params.ATAC_PLOT_METADATA_SCRIPT='atac_generate_metadata_barcode_rank_plot.R'
params.ATAC_PLOT_METADATA_HELPER_SCRIPT='barcode_rank_functions.R'

// debug file
// params.SC_ATAC_QC_BARCODE_METADATA_FILE='{nf_data_base_path}/{nf_data_project}/barcode_metadata.txt'
params.ATAC_QC_FRAGMENT_CUTOFF=10
params.SC_ATAC_QC_BARCODE_OUTPUT_FILE='atac_barcode_metadata_output_file.png'
    
// params.SC_ATAC_BARCODE_METADATA_SCRIPT='atac_generate_merged_barcode_metadata.sh'
// DEBUG: get the data from a file
// params.SC_ATAC_BARCODE_METADATA_FILTERED_BARCODE_STATS='{nf_data_base_path}nf_wf_fg_singlecell/filtered_barcode_stats.csv'
// params.SC_ATAC_BARCODE_METADATA_TSS_ENRICHMENT_BARCODE_STATS='{nf_data_base_path}nf_wf_fg_singlecell/tss_enrichment_barcode_stats.csv'

// params.SC_ATAC_GENERATE_BARCODE_RANK_PLOT_PKR='PKR123'

// params.SC_JOINT_CELL_BARCODE_PLOT_SCRTIP_PY='joint_cell_plotting.py'
// params.SC_JOINT_CELL_BARCODE_PLOT_SCRTIP_R='joint_cell_plotting_density.R'

// DEBUG

params.CPUS_TO_USE_TSS=16
params.CPUS_TO_USE_CHROMAP_IDX=32
""")


In [44]:
# !rm -r /Users/eilaarich-landkof-stanford/Documents/Code/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_atac/work
# !rm -r /home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_atac/work
# !gsutil rm -r gs://eila-nextflow-bucket/work/ATAC

In [45]:
# !rm .nextflow*

# !ls /Users/eilaarich-landkof-stanford/Documents/Code/atomic-workflows/nextflow/nf_data/test_atac_pipeline/local_atac_fastqs_files.tsv

In [ ]:
!nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow_atac.config -profile google_batch_profile -process.echo -with-tower -resume 
# -with-dag -resume

# !nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow_vm.config -profile google_batch_profile


# !nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow_vm.config -profile google_batch_profile
# !nextflow run nf_
# 
# wf_fg_singlecell_atac.nf -c nextflow_vm_anvil.config -profile google_batch_profile
# !nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow_local.config -profile google_batch_profile
# !nextflow run nf_wf_fg_singlecell_atac.nf -c nextflow_local.config -profile docker_profile


N E X T F L O W  ~  version 23.10.1
Launching `nf_wf_fg_singlecell_atac.nf` [small_morse] DSL2 - revision: c0338550c1
/home/eila/git_igvf/atomic-workflows/nextflow/nf_data/wdl_atac_pipeline/vm_atac_fastqs_files.tsv
null
after genome_fasta_ch
[-        ] process > run_zcat               -
[-        ] process > run_create_chromap_idx -
after run_zcat 
after run_create_chromap_idx

[-        ] process > run_zcat                       -
[-        ] process > run_create_chromap_idx         -
[-        ] process > run_chromap_map_to_idx         -
[-        ] process > run_add_subpool_to_chromap_... -
[-        ] process > run_bgzip_on_chromap_fragme... -
[-        ] process > run_process_create_temp_sum... -
[-        ] process > run_filter_aligned_fragment... -
[-        ] process > run_gzip_on_genes_gtf          -
[-        ] process > run_calculate_tss_enrichmen... -
[-        ] process > run_barcode_metadata_stats     -
[-        ] process > run_atac_barcode_rank_plot     -
[-        ] p

In [ ]:
!cat .nextflow.log